# Experiment and Registry the best model

In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error



In [3]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment("nyc-taxi-expertiment")

<Experiment: artifact_location='/workspaces/mlops/02-experiment-tracking/mlruns/1', creation_time=1724716763779, experiment_id='1', last_update_time=1724716763779, lifecycle_stage='active', name='nyc-taxi-expertiment', tags={}>

To run: mlflow ui --port 8080 --backend-store-uri sqlite:///mlruns.db

# Getting the data

In [3]:
mlflow.set_experiment("nyc-taxi-expertiment")
experiment = mlflow.get_experiment_by_name("nyc-taxi-expertiment")

In [4]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

# Hyperparameter tunning and tracking with Hyperopt and MLFlow

In [9]:
import xgboost as xgb
from mlflow import MlflowClient


In [10]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from hyperopt.pyll import stochastic

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [12]:
#mlflow.xgboost.autolog(log_models=True)

def print_auto_logged_info(r):
    tags = {k: v for k, v in r.data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in MlflowClient().list_artifacts(r.info.run_id, "model")]
    print(f"run_id: {r.info.run_id}")
    print(f"artifacts: {artifacts}")
    print(f"params: {r.data.params}")
    print(f"metrics: {r.data.metrics}")
    print(f"tags: {tags}")


In [13]:
def objective(params):
    with mlflow.start_run() as run:
        mlflow.set_tag("model", "xgboost")
        mlflow.set_tag('developer','Willian Lima')
        mlflow.log_param('train-path','https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
        mlflow.log_param('valid-path','https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        mlflow.xgboost.log_model(booster,artifact_path='model')
        #print_auto_logged_info(mlflow.get_run(run_id=run.info.run_id))
    return {'loss': rmse, 'status': STATUS_OK}

hyperopt.fmin(). It covers how to write an objective function that fmin can optimize, and how to describe a search space that fmin can search.

Hyperopt's job is to find the best value of a scalar-valued, possibly-stochastic function over a set of possible arguments to that function. Whereas many optimization packages will assume that these inputs are drawn from a vector space, Hyperopt is different in that it encourages you to describe your search space in more detail. By providing more information about where your function is defined, and where you think the best values are, you allow algorithms in hyperopt to search more efficiently.

I'd recommend hyperopt instead of scikit-learn's GridSearchCV. Hyperopt can search the space with Bayesian optimization using hyperopt.tpe.suggest. It will arrive at good parameters faster than a grid search and you can limit the number of iterations no matter the space size, so it's definitely better for large spaces.

In [14]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

[16:24:07] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.                                                        
[0]	validation-rmse:18.53752                                                                                                                                                  
[1]	validation-rmse:16.32192                                                                                                                                                  
[2]	validation-rmse:14.48288                                                                                                                                                  
[3]	validation-rmse:12.96282                                                                                                                                                  
[4]	validation-rmse:11.71843                                                                                                 

[91]	validation-rmse:6.53342                                                                                                                                                  
[92]	validation-rmse:6.53277                                                                                                                                                  
[93]	validation-rmse:6.53240                                                                                                                                                  
[94]	validation-rmse:6.53171                                                                                                                                                  
[95]	validation-rmse:6.53146                                                                                                                                                  
[96]	validation-rmse:6.53076                                                                                                 

[183]	validation-rmse:6.49012                                                                                                                                                 
[184]	validation-rmse:6.48967                                                                                                                                                 
[185]	validation-rmse:6.48915                                                                                                                                                 
[186]	validation-rmse:6.48871                                                                                                                                                 
[187]	validation-rmse:6.48841                                                                                                                                                 
[188]	validation-rmse:6.48777                                                                                                

[275]	validation-rmse:6.46106                                                                                                                                                 
[276]	validation-rmse:6.46068                                                                                                                                                 
[277]	validation-rmse:6.46040                                                                                                                                                 
[278]	validation-rmse:6.46035                                                                                                                                                 
[279]	validation-rmse:6.46025                                                                                                                                                 
[280]	validation-rmse:6.45978                                                                                                

[367]	validation-rmse:6.43833                                                                                                                                                 
[368]	validation-rmse:6.43819                                                                                                                                                 
[369]	validation-rmse:6.43809                                                                                                                                                 
[370]	validation-rmse:6.43782                                                                                                                                                 
[371]	validation-rmse:6.43771                                                                                                                                                 
[372]	validation-rmse:6.43766                                                                                                

...

[16:37:07] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.                                                        
[0]	validation-rmse:11.43201                                                                                                                                                  
[1]	validation-rmse:8.07133                                                                                                                                                   
[2]	validation-rmse:7.06712                                                                                                                                                   
[3]	validation-rmse:6.76509                                                                                                                                                   
[4]	validation-rmse:6.65449                                                                                                  

[91]	validation-rmse:6.40880                                                                                                                                                  
[92]	validation-rmse:6.40850                                                                                                                                                  
[93]	validation-rmse:6.40876                                                                                                                                                  
[94]	validation-rmse:6.40766                                                                                                                                                  
[95]	validation-rmse:6.40647                                                                                                                                                  
[96]	validation-rmse:6.40597                                                                                                 

[183]	validation-rmse:6.38076                                                                                                                                                 
[184]	validation-rmse:6.38065                                                                                                                                                 
[185]	validation-rmse:6.38002                                                                                                                                                 
[186]	validation-rmse:6.37969                                                                                                                                                 
[187]	validation-rmse:6.37938                                                                                                                                                 
[188]	validation-rmse:6.37953                                                                                                

2024/09/01 16:37:27 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [13:20<00:00, 80.02s/trial, best loss: 6.331972395461963]


# Interacting with the MLflow tracking server
The MlflowClient object allows us to interact with...

an MLflow Tracking Server that creates and manages experiments and runs.
an MLflow Registry Server that creates and manages registered models and model versions.
To instantiate it we need to pass a tracking URI and/or a registry URI

In [15]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

Showing existing experiments

In [16]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

list_experiments = client.search_experiments()

for experiment in list_experiments:
    print(f"Name: {experiment.name}, Id: {experiment.experiment_id}")

Name: nyc-taxi-expertiment, Id: 1


Getting the top 5 better runs

In [17]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [18]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")
    #print(run)

run id: 94ea5c6495d341f9a9af52b1689f9a8f, rmse: 6.3320
run id: 92ba14afa3b04594b94ff741394ebed7, rmse: 6.3360
run id: 6afee277872d44cf8b147f8a34c0d32e, rmse: 6.3750
run id: a0f2851d382448a3b6e7b2229da9bccd, rmse: 6.3817
run id: e34eb2f0e53348c2b7a9016d3ad623fc, rmse: 6.3825


In [19]:
# get the best model ID
best_model_id = runs[0].info.run_id
best_model_rmse = runs[0].data.metrics['rmse']


# Interacting with the Model Registry
In this section We will use the MlflowClient instance to:

Register a new version for the best experiment nyc-taxi-regressor


In [20]:
# List all registered models
registered_models = client.search_registered_models()

# Print out model names
for model in registered_models:
    print(f"Model Name: {model.name}")
    
model_name = "nyc-taxi-regressor"

Model Name: nyc-taxi-regressor


In [21]:
# Register the model
model_uri = f"runs:/{best_model_id}/model"
model_info = mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")
# Get the version of the new best model
model_version = model_info.version

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


Verify if the RSME of the new model is better than the model already in the alias "champion"

In [22]:
try:
    # get a model version by alias
    run_id = client.get_model_version_by_alias(model_name, "champion").run_id
    run = client.get_run(run_id)
    experiment_id = run.info.experiment_id
    experiment = client.get_experiment(experiment_id)

    # Print out experiment details
    print(f"Experiment ID: {experiment.experiment_id}")
    print(f"Name: {experiment.name}")
    print(f"Artifact Location: {experiment.artifact_location}")
    print(f"Lifecycle Stage: {experiment.lifecycle_stage}")

    # Print out run details
    print(f"Run ID: {run.info.run_id}")
    print(f"Start Time: {run.info.start_time}")
    print(f"End Time: {run.info.end_time}")
    print(f"Status: {run.info.status}")
    print(f"Parameters: {run.data.params}")
    print(f"Metrics: {run.data.metrics}")
    print(f"Tags: {run.data.tags}")

    current_model_rsme = run.data.metrics['rmse']
except:
    print("There is no champion still!")

Experiment ID: 1
Name: nyc-taxi-expertiment
Artifact Location: /workspaces/mlops/02-experiment-tracking/mlruns/1
Lifecycle Stage: active
Run ID: e34eb2f0e53348c2b7a9016d3ad623fc
Start Time: 1725205968285
End Time: 1725205981826
Status: FINISHED
Parameters: {'train-path': 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet', 'valid-path': 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet', 'learning_rate': '0.2968892075721066', 'max_depth': '27', 'min_child_weight': '1.5623975963187875', 'objective': 'reg:linear', 'reg_alpha': '0.0329752054501982', 'reg_lambda': '0.11058764164101563', 'seed': '42'}
Metrics: {'rmse': 6.382490392331548}
Tags: {'mlflow.user': 'codespace', 'mlflow.source.name': '/home/codespace/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.runName': 'peaceful-fowl-605', 'model': 'xgboost', 'developer': 'Willian Lima', 'mlflow.log-model.history': '[{"run_id": "e34

In [23]:
try:
    current_model_version = client.get_model_version_by_alias(model_name, "champion").version
    # If the new model is better than the current champion, replace it
    if best_model_rmse <= current_model_rsme:
        # Set model version tag
        # Seting this new model to validation pending
        client.set_model_version_tag(model_name, model_version, "validation_status", "pending")
        # Seting a alias to this model champion to show that is the best from this experiment
        client.set_registered_model_alias(model_name, "champion", model_version)
        print(f"The new model is better than the current champion! Congratz!! Replacing the allias champion to the new model. Now, the version {model_version}")

    else:
        print(f"The new version os not better than the champion, keeping the current champion. The current champion is still {current_model_version}")
except:
    # If the new model is better than the current champion, replace it
    if best_model_rmse <= current_model_rsme:
        # Set model version tag
        # Seting this new model to validation pending
        client.set_model_version_tag(model_name, model_version, "validation_status", "pending")
        # Seting a alias to this model champion to show that is the best from this experiment
        client.set_registered_model_alias(model_name, "champion", model_version)
        print(f"The new model is better than the current champion! Congratz!! Replacing the allias champion to the new model. Now, the version {model_version}")

    else:
        print(f"The new version os not better than the champion, keeping the current champion. There is no champion still")


The new model is better than the current champion! Congratz!! Replacing the allias champion to the new model. Now, the version 3


In [24]:
# Retrieving information about the model by alias

champion_model_version = client.get_model_version_by_alias(model_name, "champion").version

In [25]:
f"models:/{model_name}/{model_version}"

'models:/nyc-taxi-regressor/3'

Retriving the ''champion'' model and runing it

In [26]:
client.search_registered_models()


[<RegisteredModel: aliases={'champion': 3}, creation_timestamp=1725206243583, description=None, last_updated_timestamp=1725208647476, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1725208647476, current_stage='None', description=None, last_updated_timestamp=1725208647476, name='nyc-taxi-regressor', run_id='94ea5c6495d341f9a9af52b1689f9a8f', run_link=None, source='/workspaces/mlops/02-experiment-tracking/mlruns/1/94ea5c6495d341f9a9af52b1689f9a8f/artifacts/model', status='READY', status_message=None, tags={'validation_status': 'pending'}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>]

In [27]:
model_version

3

In [28]:
import mlflow.pyfunc

model_name = "nyc-taxi-regressor"
model_version = champion_model_version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

model.predict(X_val)


[16:37:27] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


array([14.510142 ,  7.2604923, 14.555611 , ..., 13.587127 ,  6.087927 ,
        8.160798 ], dtype=float32)